<a href="https://colab.research.google.com/github/brianthomaslewis/polygence_work/blob/main/Movie_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 2.1 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149865 sha256=62f75283c5480c466d293c53c8d6b0d7b30ab97576b8ade494b545b0c0b4650d
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [2]:
import os
import pandas as pd
from zipfile import ZipFile
from fuzzywuzzy import fuzz
from itertools import chain
from io import BytesIO
from urllib.request import urlopen
import ipywidgets as widgets
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# Read data from dataset online directly
resp = urlopen('https://files.grouplens.org/datasets/movielens/ml-25m.zip')
zf = ZipFile(BytesIO(resp.read()))
for name in zf.namelist():
  print(name)

ml-25m/
ml-25m/tags.csv
ml-25m/links.csv
ml-25m/README.txt
ml-25m/ratings.csv
ml-25m/genome-tags.csv
ml-25m/genome-scores.csv
ml-25m/movies.csv


In [4]:
ratings = pd.read_csv(zf.open('ml-25m/ratings.csv'))
movies = pd.read_csv(zf.open('ml-25m/movies.csv'))
tags = pd.read_csv(zf.open('ml-25m/tags.csv'))
links = pd.read_csv(zf.open('ml-25m/links.csv'))
genome_scores = pd.read_csv(zf.open('ml-25m/genome-scores.csv'))


In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
import re

def clean_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title

In [7]:
movies["clean_title"] = movies["title"].apply(clean_title)

In [8]:
movies

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama,We 2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(movies["clean_title"])

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices].iloc[::-1]
    
    return results

In [11]:
import ipywidgets as widgets
from IPython.display import display

movie_input = widgets.Text(
    value = "Toy Story",
    description = "Movie Title:",
    disabled = False
)
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) > 3:
            display(search(title))

movie_input.observe(on_type, names = 'value')


display(movie_input, movie_list)

Text(value='Toy Story', description='Movie Title:')

Output()

In [12]:
movie_id = 89745
movie = movies[movies["movieId"] == movie_id]

In [13]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [14]:
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()

In [15]:
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]

In [16]:
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

similar_user_recs = similar_user_recs[similar_user_recs > .10]

In [17]:
all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]

In [18]:
all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())


In [19]:
rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
rec_percentages.columns = ["similar", "all"]

In [20]:
rec_percentages

,similar,all
1,0.236083,0.126250
32,0.103877,0.101516
47,0.203115,0.146232
50,0.211067,0.202959
110,0.182240,0.162835
...,...,...
134853,0.198641,0.036444
152081,0.133532,0.020652
164179,0.128728,0.029124
166528,0.124751,0.014411


In [21]:
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]


In [22]:
rec_percentages = rec_percentages.sort_values("score", ascending=False)

In [23]:
rec_percentages

,similar,all,score
89745,1.000000,0.040459,24.716368
106072,0.103711,0.005289,19.610199
122892,0.241054,0.012367,19.491770
102125,0.216534,0.012119,17.867419
88140,0.215043,0.012052,17.843074
...,...,...,...
296,0.288933,0.288146,1.002730
593,0.222830,0.228665,0.974483
527,0.199967,0.217833,0.917984
1193,0.100895,0.120244,0.839081


In [24]:
rec_percentages.head(10).merge(movies, left_index=True, right_on = "movieId")



,similar,all,score,movieId,title,genres,clean_title
17067,1.000000,0.040459,24.716368,89745,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX,Avengers The 2012
20513,0.103711,0.005289,19.610199,106072,Thor: The Dark World (2013),Action|Adventure|Fantasy|IMAX,Thor The Dark World 2013
25058,0.241054,0.012367,19.491770,122892,Avengers: Age of Ultron (2015),Action|Adventure|Sci-Fi,Avengers Age of Ultron 2015
19678,0.216534,0.012119,17.867419,102125,Iron Man 3 (2013),Action|Sci-Fi|Thriller|IMAX,Iron Man 3 2013
16725,0.215043,0.012052,17.843074,88140,Captain America: The First Avenger (2011),Action|Adventure|Sci-Fi|Thriller|War,Captain America The First Avenger 2011
16312,0.175447,0.010142,17.299824,86332,Thor (2011),Action|Adventure|Drama|Fantasy|IMAX,Thor 2011
21348,0.287608,0.016737,17.183667,110102,Captain America: The Winter Soldier (2014),Action|Adventure|Sci-Fi|IMAX,Captain America The Winter Soldier 2014
25071,0.214049,0.012856,16.649399,122920,Captain America: Civil War (2016),Action|Sci-Fi|Thriller,Captain America Civil War 2016
25061,0.136017,0.008573,15.865628,122900,Ant-Man (2015),Action|Adventure|Sci-Fi,AntMan 2015
14628,0.242876,0.015517,15.651921,77561,Iron Man 2 (2010),Action|Adventure|Sci-Fi|Thriller|IMAX,Iron Man 2 2010


In [63]:
def find_similar_movies(movie_id, genre_filter):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
    similar_user_recs = similar_user_recs[similar_user_recs > .10]

    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)

    # ADDITION 1: Changed the filtering of the results to incorporate genre filters
    unfiltered_df = rec_percentages.merge(movies, left_index=True, right_on = "movieId")[["score", "title", "genres"]]
    filtered_df = unfiltered_df[unfiltered_df.genres.str.contains(genre_filter)].head(10)
    filtered_df.genres = filtered_df.genres.str.replace('|',', ', regex=True)
    
    return filtered_df

# find_similar_movies(89745, 'Action') # Test the function with The Avengers and an added genre filter

In [67]:
# Genre-based filtering additions

# Create distinct list of genres
unique_genres = sorted(list(set(list(chain(*[i.split('|') for i in movies.genres.unique().tolist()])))))
unique_genres.pop(0) # Get rid of "no genres listed" element in list

# create an object for TfidfVectorizer
tfidf_vector = TfidfVectorizer(stop_words='english')
# apply the object to the genres column
tfidf_matrix = tfidf_vector.fit_transform(unique_genres)

from sklearn.metrics.pairwise import linear_kernel
# create the cosine similarity matrix
sim_matrix = linear_kernel(tfidf_matrix,tfidf_matrix)

# Function to find most similar genre
def find_closest_genre(genre):
   leven_scores = [(element, fuzz.ratio(element, genre)) for element in unique_genres]
   sorted_leven_scores = sorted(leven_scores, key=lambda x: x[1], reverse=True)
   closest_genre = sorted_leven_scores[0][0]
   distance_score = sorted_leven_scores[0][1]
   return closest_genre, distance_score   

In [69]:
import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display

w_genre = widgets.Dropdown(
    options = sorted(unique_genres), 
    value = 'Children'
)

w_title = widgets.Text(
    value = 'Toy Story',
    description = 'Movie Title:',
    disabled = False
)

def movie_recommender(genre, title):
    if len(title) > 3:
        results = search(title)
        movie_id = results.iloc[0]["movieId"]
        display(find_similar_movies(movie_id, genre))

w_box = interactive(movie_recommender, genre=w_genre, title=w_title)

display(w_box)

interactive(children=(Dropdown(description='genre', index=3, options=('Action', 'Adventure', 'Animation', 'Chi…